In [1]:
import polars as pl
import altair as alt

In [2]:
pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [3]:
df = pl.read_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1918
).filter(
    ~(pl.col("komora_komplet").str.contains("Senát") 
      | 
      pl.col('komora_komplet').str.contains('Společn') 
      | 
      pl.col('komora_komplet').str.contains('Ústavod') 
      | 
      pl.col('komora').str.contains('psse')
     )
).drop_nulls(
    subset=['datum']
)

In [4]:
df.select(pl.col("komora")).unique().to_series().to_list()

['ns', 'ps', 'cnr', 'pns', 'fs']

In [5]:
df.filter(pl.col("komora").str.contains("psse"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool


In [6]:
df.select(pl.col("komora_komplet")).unique().to_series().to_list()

['Česká národní rada 1969 – 1971',
 'Federální shromáždění ČSFR, 1990 – 1992 Sněmovna lidu',
 'PČR, PS 2021-...',
 'Parlament České republiky, Poslanecká sněmovna 2006 – 2010',
 'FS ČSSR 1969-1971',
 'PČR, PS 2002-2006',
 'NS RČS 1960-1964',
 'Parlament České republiky, Poslanecká sněmovna 1996 – 1998',
 'Federální shromáždění ČSSR, 1981 – 1986 Sněmovna národů',
 'ČNR 1968',
 'Federální shromáždění ČSSR, 1981 – 1986 Sněmovna lidu',
 'Česká národní rada 1986 – 1990',
 'Parlament České republiky, Poslanecká sněmovna 2010 – 2013',
 'PČR, PS 2017-...',
 'Federální shromáždění ČSSR, 1986 – 1990 Sněmovna národů',
 'Národní shromáždění československé / Poslanecká sněmovna1920 – 1925, Poslanecká sněmovna',
 'PČR, PS 2006-2010',
 'NS RČS 1935-1938, Poslanecká sněmovna',
 'Parlament České republiky, Poslanecká sněmovna 1998 – 2002',
 'NS RČS 1964-1968',
 'Česká národní rada 1990 – 1992',
 'Národní shromáždění československé 1918 – 1920',
 'Česká národní rada 1992 – 1992',
 'Parlament České repub

In [7]:
df.sample()

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool
"""Poslanec Vlastimil Válek""","""2017ps_stenprot_049schuz_s049119.htm""","""/sqw/detail.sqw?id=6482""","""Děkuji, pane předsedající. Teď si nejsem jistý, jestli COVID se nešíří Poslaneckou sněmovnou anebo jestli je to jaksi reakce na texaský masakr motorov…",1,"""ps""",2017,49,119,2020-05-28,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""49. schůze""",true,true,true


In [8]:
import re

In [86]:
def najdi_lexy(retezec):
    return [x.replace('lex','Lex').replace('Ukrajinu','Ukrajina').replace('Ukrajině','Ukrajina').replace('Ukrajinou','Ukrajina').replace('Ukrajiny','Ukrajina').replace('plynu','plyn') for x in re.findall(r'[Ll]ex \w{1,20}-{0,1}\w{1,20}',retezec)]

In [41]:
najdi_lexy('Přijali jsme lex Babiš, pak Lex Švancnberk, lex Pat-Mat, to bylo z lexů vše.')

['Lex Babiš', 'Lex Švancnberk', 'Lex Pat-Mat']

In [42]:
lexy = df.filter(pl.col('text').str.contains(r' [Ll]ex ')).with_columns(pl.col("text").map_elements(najdi_lexy).alias("lex")).explode('lex').group_by("lex").len().sort(by="len",descending=True)
lexy

C:\Users\micha\AppData\Local\Temp\ipykernel_16968\901567343.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  lexy = df.filter(pl.col('text').str.contains(r' [Ll]ex ')).with_columns(pl.col("text").map_elements(najdi_lexy).alias("lex")).explode('lex').group_by("lex").len().sort(by="len",descending=True)


lex,len
str,u32
"""Lex Ukrajina""",600
"""Lex Babiš""",169
"""Lex specialis""",86
"""Lex OZE""",62
"""Lex ČEZ""",32
"""Lex Dukovany""",31
"""Lex Cyril""",27
"""Lex esto""",22
"""Lex covid""",22


In [80]:
pomluvcich = df.filter(pl.col("obdobi") == 2021).filter(pl.col('text').str.contains(r' [Ll]ex ')).with_columns(pl.col("text").map_elements(najdi_lexy).alias("lex")).explode('lex').group_by("lex").agg(pl.col("mluvci").unique().len()).sort(by="mluvci",descending=True)
pomluvcich

C:\Users\micha\AppData\Local\Temp\ipykernel_16968\2506151919.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  pomluvcich = df.filter(pl.col("obdobi") == 2021).filter(pl.col('text').str.contains(r' [Ll]ex ')).with_columns(pl.col("text").map_elements(najdi_lexy).alias("lex")).explode('lex').group_by("lex").agg(pl.col("mluvci").unique().len()).sort(by="mluvci",descending=True)


lex,mluvci
str,u32
"""Lex Ukrajina""",63
"""Lex OZE""",20
"""Lex specialis""",13
"""Lex plyn""",11
"""Lex ČEZ""",7
"""Lex Babiš""",7
"""Lex Národní""",3
"""Lex Pawlas""",3
"""Lex Vrbětice""",3


In [84]:
pomluvcich.filter(pl.col("mluvci") >= 2)

lex,mluvci
str,u32
"""Lex Ukrajina""",63
"""Lex OZE""",20
"""Lex specialis""",13
"""Lex plyn""",11
"""Lex ČEZ""",7
"""Lex Babiš""",7
"""Lex Národní""",3
"""Lex Pawlas""",3
"""Lex Vrbětice""",3


In [43]:
lexy21 = df.filter(pl.col("obdobi") == 2021).filter(pl.col('text').str.contains(r' [Ll]ex ')).with_columns(pl.col("text").map_elements(najdi_lexy).alias("lex")).explode('lex').group_by("lex").len().sort(by="len",descending=True)
lexy.rename({"len":"n"})

C:\Users\micha\AppData\Local\Temp\ipykernel_16968\2386524876.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  lexy21 = df.filter(pl.col("obdobi") == 2021).filter(pl.col('text').str.contains(r' [Ll]ex ')).with_columns(pl.col("text").map_elements(najdi_lexy).alias("lex")).explode('lex').group_by("lex").len().sort(by="len",descending=True)


lex,n
str,u32
"""Lex Ukrajina""",600
"""Lex Babiš""",169
"""Lex specialis""",86
"""Lex OZE""",62
"""Lex ČEZ""",32
"""Lex Dukovany""",31
"""Lex Cyril""",27
"""Lex esto""",22
"""Lex covid""",22


In [72]:
lexy.filter(pl.col("len") > 1)

lex,len
str,u32
"""Lex Ukrajina""",600
"""Lex Babiš""",169
"""Lex specialis""",86
"""Lex OZE""",62
"""Lex ČEZ""",32
"""Lex Dukovany""",31
"""Lex Cyril""",27
"""Lex esto""",22
"""Lex covid""",22


In [70]:
lexy.select(pl.col("len").median())

len
f64
1.0


In [44]:
lexy_top = lexy.drop_nulls().filter(pl.col('len') > 10).select(pl.col('lex')).to_series().to_list()
lexy_top

['Lex Ukrajina',
 'Lex Babiš',
 'Lex specialis',
 'Lex OZE',
 'Lex ČEZ',
 'Lex Dukovany',
 'Lex Cyril',
 'Lex esto',
 'Lex covid',
 'Lex plyn',
 'Lex Dérer',
 'Lex Perek',
 'Lex posterior',
 'Lex Baťa',
 'Lex generalis',
 'Lex Šumava',
 'Lex Uhlíř',
 'Lex Ofner']

In [64]:
poobdobi = df.with_columns(
    pl.col("text").str.count_matches("(?i) lex ").alias("lex")
).group_by("obdobi").agg(pl.col("lex").sum()).sort(by="obdobi")

poobdobi

obdobi,lex
i32,u32
1918,12
1920,56
1925,78
1929,38
1935,17
1945,2
1948,0
1954,0
1960,0


In [66]:
poobdobi.filter(pl.col('obdobi') < 2020).select(pl.col('lex').sum())

lex
u32
541


In [46]:
def hledej(text, n=3, delka=200):
    nalezy = df.sort(by="datum").filter(pl.col("text").str.contains(text)).head(n).iter_rows()
    for cislo, radek in enumerate(nalezy):
        print(f'Kdo: {radek[0]}')
        print(f'Kde: {radek[1]}')
        print(f'Kdy: {radek[9]}')
        
        # Split the text around the search term
        parts = re.split(text, radek[3])
        
        # Get the part before the search term (safe if empty)
        before = parts[0] if len(parts) > 0 else ""
        # Get the part after the search term (safe if empty)
        after = parts[1] if len(parts) > 1 else ""
        
        # Take up to 200 characters from before and after, but don't break if shorter
        before_snippet = before[-delka:] if len(before) > delka else before
        after_snippet = after[:delka] if len(after) > delka else after
        nalez = re.search(text, radek[3]).group()
        
        uryvek = before_snippet + nalez + after_snippet
        print(f'Co: {uryvek}')
        
        if cislo != 2:  # Changed from 3 to 2 since enumerate starts at 0
            print("***")

In [47]:
hledej("lex lečo")

Kdo: Poslanec Hubert Lang
Kde: 2021ps_stenprot_055schuz_s055125.htm
Kdy: 2023-03-01
Co: parlamentní kulturu. Já jsem dneska tedy ve dvě hodiny ráno skončil, měl jsem tady nějaké faktické poznámky, protože nejsem právník, tak se nemůžu vyjadřovat k právnímu stavu věci. Probrali jsme tady lex lečo, že jo, například, a další věci. Před nějakým časem jsem vysílal signály k panu premiérovi na Úřad vlády, který je nepřijal. Já prostě také nesouhlasím s tím, aby mi někdo omezil mojí jedinou šanci, 
***


In [48]:
hledej("lex dvojčata")

Kdo: Poslanec Šimon Heller
Kde: 2021ps_stenprot_078schuz_s078263.htm
Kdy: 2023-10-27
Co: Dámy a pánové, budu velmi krátký. Všechny vás chci naléhavě požádat o podporu lex dvojčata. Je to pozměňovací návrh, který napravuje historickou křivdu, kdy dneska dvojčata mají nárok pouze na 1,5násobek příspěvku rodičovské. Je to obrovská historická křivda. A tato Sněmovna má podruhé mož
***
Kdo: Poslanec Šimon Heller
Kde: 2021ps_stenprot_136schuz_s136036.htm
Kdy: 2025-04-16
Co: Dámy a pánové, obracím se na vás (Hlasitě.) - doufám, že jsem vás nevylekal - s návrhem, teď vás vylekám možná tím pozměňovacím návrhem, který se týká dvojčat, lex dvojčata. Jsem přesvědčen, že bychom měli narovnat práva dvojčat, a proto jsem načetl pozměňovací návrh, který napravuje historickou nespravedlnost, která tady je, protože stále stát považuje dvojčata za 1,5n
***
Kdo: Poslanec Šimon Heller
Kde: 2021ps_stenprot_136schuz_s136040.htm
Kdy: 2025-04-16
Co: Děkuji za slovo. Hlásím se k pozměňovacímu návrhu číslo 6473 

In [49]:
hledej("lex železářství")

Kdo: Poslanec Jakub Michálek
Kde: 2021ps_stenprot_041schuz_s041122.htm
Kdy: 2022-10-14
Co: ě Kocmanové zajišťuje, že bude vhodně implementován mechanismus filtrování, o němž samotném jsme vyjádřili určité pochybnosti. Další návrh, který jsme předložili s kolegou Hellerem, to je to takzvané lex železářství, nebo kdo chce kadeřnictví, nebo pan ministr má prodavačku, tak to je návrh, který skutečně navazuje na judikaturu Soudního dvora Evropské unie. Musím zde reagovat na obavy, které tady zazněly, že te
***


In [50]:
hledej("lex Brno")

Kdo: Poslanec Ivan Adamec
Kde: 2021ps_stenprot_053schuz_s053133.htm
Kdy: 2023-03-07
Co: o místní rozvoj, ty pozměňovací návrhy byly staženy anebo se neodsouhlasily. Na druhé straně, už jsem tam říkal, že mě trápí ještě jedna věc, a to je vyřešit, řeknu to velmi jednoduše, lex Dukovany a lex Brno, to znamená tu trubku z Dukovan do Brna, protože to je záležitost, ta myšlenka už je letitá, ale díky situaci, která tady nastala po ruské invazi a po energetické krizi, je ta myšlenka velmi aktuální
***
